In [30]:
x = raw.copy()
x.set_montage(make_standard_montage("standard_alphabetic"))
x.plot_sensors(ch_type="eeg");

ValueError: DigMontage is only a subset of info. There are 33 channel positions not present in the DigMontage. The channels missing from the montage are:

['FC5', 'FC1', 'FC2', 'FC6', 'T7', 'T8', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P8', 'POz', 'AF7', 'AF3', 'AF4', 'AF8', 'FC3', 'FCz', 'FC4', 'CP3', 'CPz', 'CP4', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.

In [77]:
from mne.io import read_raw_eeglab, read_raw
from mne.channels import get_builtin_montages, make_standard_montage
raw = read_raw_eeglab("../sub400/400MNE.set")
raw = raw.crop(tmin=60, tmax=60+5*60)
raw.set_channel_types({"BIP1":"eog","BIP2":"misc","BIP3":"misc","BIP4":"misc", "AUX1":"misc","AUX2":"misc","AUX3":"misc","AUX4":"misc", "M1":"misc", "M2":"misc"})

Reading /home/dorian/PycharmProjects/internship/learning_eeg/../sub400/400MNE.fdt


/tmp/ipykernel_15627/778817812.py:3: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = read_raw_eeglab("../sub400/400MNE.set")
/tmp/ipykernel_15627/778817812.py:5: RuntimeWarning: The unit for channel(s) AUX1, AUX2, AUX3, AUX4, BIP2, BIP3, BIP4, M1, M2 has changed from V to NA.
  raw.set_channel_types({"BIP1":"eog","BIP2":"misc","BIP3":"misc","BIP4":"misc", "AUX1":"misc","AUX2":"misc","AUX3":"misc","AUX4":"misc", "M1":"misc", "M2":"misc"})


<RawEEGLAB | 400MNE.fdt, 72 x 307201 (300.0 s), ~85 kB, data not loaded>

In [2]:
import matplotlib.pyplot as plt
from ipywidgets import Textarea, Text, Label, Button, VBox, HBox, HTML, Image, Tab, Output, Layout
from IPython.display import display, clear_output
import numpy as np
import markdown

In [71]:
def get_answer_to_reveal(text, answer, with_input_field=False):
    test_reveal_button = Button(description="Check answer")
    
    test_answer = HTML(answer)
    test_answer.layout.visibility="hidden"
    def show_answer(b):
        test_answer.layout.visibility="visible"
    test_reveal_button.on_click(show_answer)
    layout = Layout(width="50%", border="solid purple")
    if with_input_field:
        elements = VBox([HTML(text), HBox([Text(), test_reveal_button]), test_answer], layout=layout)
    else:
        elements = VBox([HTML(text), HBox([test_reveal_button]), test_answer], layout=layout)
    return elements
    
def get_chapter(panels, panel_titles):
    tab = Tab()
    tab.children = panels
    tab.titles = panel_titles
    chapter = tab
    return chapter

In [72]:
def code_block(code):
    text = f"""
    <style>
      pre {{
        background-color: #f5f5f5;
        padding: 10px;
        border-radius: 5px;
        overflow-x: auto;
      }}
      code {{
        font-family: monospace;
        color: #333;
    }}
    </style>
    <pre><code>
    {code}
    </code></pre>
    """
    return text


In [73]:
def check_data(d):
    ch_names = d.ch_names
    assert type(ch_names) == list and len(ch_names) > 0
    assert raw.n_times > 60
    

# Start

In [87]:
raw = None

text = """
Welcome! 
This tutorial has two purposes
<br>
<b>First</b>, it teaches you to work with EEG data with explanations and visualizations. 
<br>
<b>Second</b>, it allows you to explore your own data along the explanations, to directly apply your new knowledge. This tutorial can best be used, if you have an EEG dataset at hand. 
If you have one, you will read the data in the following. You don't need to know anything about the data yet. You will learn it during this course. 
If you don't have any dataset, you can still continue with the tutorial. You will be missing some interactions and tests, but you can still read all the texts and learn. 

"""
panel_1 = HTML(text)

text = """
Do you have an EEG dataset?
"""
output = Output()
output_2 = Output()
yes_button = Button(description="yes")
no_button = Button(description="No")
def no(b):
    text = "Okay. Just continue with the next panel."
    with output:
        display(HTML(text))
        
def read_data(path):
    global raw
    raw = read_raw(path)
    raw = raw.crop(tmin=60, tmax=60+5*60)
    raw.set_channel_types({"BIP1":"eog","BIP2":"misc","BIP3":"misc","BIP4":"misc", "AUX1":"misc","AUX2":"misc","AUX3":"misc","AUX4":"misc", "M1":"misc", "M2":"misc"})


def yes(b):
    input_text = Text()

    def display_read_data(b):
        path = input_text.value
        try:
            read_data(path)
            try:
                check_data_plausibility(raw)
            except AssertionError as e:
                with output_2:
                    clear_output()
                    display(HTML("The data has been read, but it does not look plausible. Are you sure that this is the correct file? You can continue with it, but it might fail at a later stage."))
            with output_2:
                clear_output()
                display(HTML("The file has been read succesfully. You can continue!"))
        except Exception as e:
            with output_2:
                clear_output()
                display(HTML("There was a problem with this file: <br>" + str(e)))
    
    text = "Great! Past the path to your datafile here:"
    read_in_button = Button(description="Read Data")
    read_in_button.on_click(display_read_data)
    
    with output:
        clear_output()
        display(VBox([HTML(text), input_text, read_in_button]))
    
no_button.on_click(no)
yes_button.on_click(yes)
panel_2 = VBox([HTML(text), HBox([yes_button, no_button]), output, output_2])


code = code_block("""
x = mne.call_function(x)
""")
text = f"""
Within this course, we will use a python-library called MNE-python from time to time. 
MNE-python provides lots of functions you can use to work with EEG data. 
When we use some code from MNE, it will be displayed like this: 
{code}

This is not a full tutorial on MNE though. If you want to learn more about MNE, you can browse their tutorials by clicking <a href="https://mne.tools/stable/documentation/index.html"><u>here</u></a>.
If you don't care about coding and using MNE, you can also ignore the code examples. You will still be able to understand what is happening.
"""

panel_3 = HTML(text)

text = """
The course is structured as follows:
<ul>
<li>Chapters (1) and (2) explain how an EEG works and how the data recorded with an EEG looks like. </li>
<li>Chapters (3) to (5) go into more detail an how periodic signals can be described and analyzed in general. These chapter are not EEG-specific. </li>
<li>Chapter (6) describes the EEG signal using the learnings from (3) to (5). </li>
<li> Chapter (7) introduces the power spectrum, which is a more sophisticated way to analyze and understand EEG data.</li>
</ul>
"""
panel_4 = HTML(text)

chapter = get_chapter([panel_1, panel_2, panel_3, panel_4], ["1","2","3", "4"])
display(chapter) #../sub400/400MNE.set

# (1) What is an EEG?

In [78]:
text = """
What is an EEG at all? EEG stands for Electroencephalography. "Encephalon" is the greek word for brain and "graphein" means writing. Accordingly, an EEG is a device that writes electric brain signals.
You may already know, that the brain sends electric signals along the neurons' axons. There are plenty of neurons and they send many tiny signals per second. If researchers want to measure these signals on the basis of single neurons, they cut open the head and implement tiny measurement tools. 
This is done in animals most often, because human's don't want their head to be cut open. 
The EEG however, doesn't need any operation beforehand, which is why we call it a non-invasive method. You can easily imagine that this makes it much more accessible for research. You just need to place an eeg cap on the participants head that may look like this:
"""
im_eeg_hood = Image(value=open("resources/EEG_cap.jpg", "rb").read(),width=200,height=250)
panel_1 = VBox([HTML(text), im_eeg_hood])


text = """
I already mentioned, that the EEG measures electric signals of neurons, and it does that from outside of the skull. 
If a single neurons sends an electric signal, we would never be able to record this from the outside of the skull, though, because it would be way too small. 
However, if many neurons send electric signals at the same time, they sum up and this can be measured from the outside with sensitive electrodes. This is what the EEG does. 
It is important to understand, that the EEG doesn't record <i>all</i> the brain activity but just the very strong highlights. 
Imagine you are standing outside of a soccer arena. You don't see the game, you just hear the fans' voices. Would you be able to understand everyting that happens within the game?
Most likely not. But you would be able to recognize major events like a scored goal, because here all fan's (or half of them) cheer altogether. 
Transferred to the brain, these are the kinds of signals an EEG measures. 
"""
panel_2 = HTML(text)

text = """
On the picture shown before you may have recognized, that an EEG cap includes multiple electrodes at different locations. 
The brain has a very sophisticated structure and different parts of it are responsible for different functions. 
For example (and strongly simplified), the back part of the prain (the so-called occpital region) is associated with processing visual information, while the frontal part is responsible for deliberate and conscious thought. 
Due to that structure, it makes sense to place electrodes at different locations on the skull to measure these differences. 
On top of that, multiple electrodes alow to measure a signal more reliably. Single electrodes will always have lots of noise in their measurement (we will discuss this later in more detail). 
If you take more electrodes, their noise will be different but the underlying signal will be caught by all of them (to a different extend) and this allows you, to extract this signal more easily. 
"""
panel_3 = HTML(text)

text = """
How many electrodes does an EEG have at all and where exactly are they located? 
There is not <i>the one and only</i> eeg cap, but there are different standards on the number and location of electrodes. 
The format used most often is known as 10-20-system. In the following you see a visualization of the electrodes.
"""
im_10_20 = Image(value=open("resources/10_20.png", "rb").read(),width=400,height=600)
text_2 = """
Do you see the nose at the top (Nasion) and the ears to the left and right? The person in the image looks upwards and you look at their head from above. 
You see that the electrodes all have names like NZ, PO3 or P9. When you look at the data later, you want to know which electrode captures which signal, and this is why you want to be familiar with the naming scheme.
The first letter(s) indicate the region of the brain the electrode is placed on: pre-frontal (Fp), frontal (F), temporal (T), parietal (P), occipital (O) and central (C). 
This indicates the position on the axis from nose to back of the head. 
On top of that, there is information on the location of the axis from ear to ear. Electrodes with a "Z" are placed in the middle of the brain. 
Electrodes with even numbers are placed on the right part of the head, while electrodes with uneven numbers are on the left part. 
For example, FZ is frontal and in the middle. P4 is parietal and on the right part. C3 is central and on the left part of the brain. 
You also see two electrodes "A1" and "A2" placed behind the ear. Sometimes they are also called "M1" and "M2" instead. 
In some picture there are also two points NZ and IZ. These are <b>no</b> electrodes, but just points of reference, that are used, when the EEG cap is placed on a person's skull. 
While they may appear on such visualizations, you will never find EEG data recorded from electrodes called NZ or IZ. 
"""
panel_4 = VBox([HTML(text), im_10_20, HTML(text_2)])

text = """
The 10-20 system we just saw had 21 electrodes in total. 
However, sometimes you don't need that much, or you need even more. Hence, different standards exist with a different numbers of electrodes. 
Often you may see the 10-10 system, which is an extension of the 10-20 system. If you observe it carefully, you will see that it includes all the electrodes of the 10-20 system, extended by some more:
"""
im_10_10 = Image(value=open("resources/10_10.png", "rb").read(),width=400,height=600)

text_2 = """
Other EEG caps may use less electrodes like the 14 ones here:
"""
im_14 = Image(value=open("resources/14_channel_eeg.png", "rb").read(),width=400,height=600)

text_3 = """
When working with EEGs, it is important to decide on the number of electrodes before, depending on the equipment available and the research's demands. 
Even more important is to know which system has been used when interpreting the data, though. 
When you analyze EEG signals (as we will do in this tutorial later), you need to know how many electrodes recorded them and where exactly these electrodes have been placed. 
"""
panel_5 = VBox([HTML(text), im_10_10, HTML(text_2), im_14])

montages = get_builtin_montages()
montages = "<br>".join(montages)
channels = np.random.choice(raw.ch_names, size=min(6, len(raw.ch_names)))
code = code_block("""
raw.set_montage(make_standard_montage("standard_1020"))
raw.plot_sensors(ch_type="eeg")
""")
text = f"""
MNE comes with a list of builtin channel systems (also called "montages") that are the following:
<br>
{montages}

Let's assume your data is missing the information on which system has been used exactly. However, you still have the channel names and from these, the system may be inferred. 
Here are some channels of your data:
{channels}
Can you already guess, which system has been used?
MNE also provides a function to apply a selected montage to a dataset. 
{code}
Try it with one of the montages listed above!
Can you find the right montage for your data?
What happens, if you use a montage that doesn't fit? 
"""
text_input = Text()
def try_out_montage(name=None):
    if name is None:
        name = text_input.value
    try:
        x = raw.copy()
        x.set_montage(make_standard_montage(name))
        return x.plot_sensors(ch_type="eeg", show=False)
    except Exception as e:
        return str(e)
output = Output()
def display_montage(b):
    montage = try_out_montage()
    with output:
        clear_output()
        display(montage)
button = Button(description="Apply Montage")
button.on_click(display_montage)

panel_6 = VBox([HTML(text), text_input, button, output])

chapter = get_chapter([panel_1, panel_2, panel_3, panel_4, panel_5, panel_6], ["1","2","3","4", "5", "Try it yourself!"])
display(chapter)

# (2) EEG Data

In [79]:
%matplotlib inline
%matplotlib widget

text = """
We now know what an EEG is. Next it is time to take a look at the data that is collected with an EEG. Most of the time, you will see the data being displayed like this: 
"""

im_eeg = Image(value=open("resources/eeg_raw_signal.png", "rb").read(),width=400,height=600)

text_2 = """
First of all, you see multiple lines of data here. These are channels and corespond to the different electordes you are already familiar with. You see the electrode/channel names on the left.
You also see, that the data is ploted over time on the x-axis. 
With an eeg, you don't just record brain activity at a given point in time, but you record it over a longer time. 
The y-axis displays a notion of strength of a signal. We will discuss this in a later chapter in more detail. 
For now, you should become familiar with the idea, that there are multiple channels with a signal varying over time each.
"""

panel_1 = VBox([HTML(text), im_eeg, HTML(text_2)])

text = """This is how your data looks like.
Note that you can interact with this visualization. You can click on the bars on the x and y axis to jump to different points in time or different channels.
Do the signals differ between the channels? Are there channels that stand out in particular?
Can you use your knowledge about the electrodes' locations to explain some of these differences?"""

output = Output()
with output:
    raw.plot(n_channels=4)

panel_2 = VBox([HTML(text), output])

chapter = get_chapter([panel_1, panel_2], ["1","Try it yourself!"])
display(chapter)

# (3) What is a periodic signal?

In [81]:
%matplotlib inline
width, height = 600, 800

im_string_moving = Image(value=open("resources/string_moving.gif", "rb").read(),width=width,height=height)

text = """
We have taken a first look at EEG data in the previous chapter. 
Now we want to understand it in more detail. To this end, we first need to take a step back and talk about what periodic signals are in general. 
In many applications you have processes that are repeating. For example, if you pull the string of a guitar and let it go, this string will move back and forth in a periodic manner.
Here is a visualization. The string starts at the top (a), then it moves back to the middle (e) and continues moving in the opposite direction (i). 
Then it moves back to the middle again (m) and back to the position it started from (p and ultimatively a). From here it moves to the middle again and so on. 


"""

text_2 = """
Now imagine you are able to measure the position of middle point of the string at different points in time during this process. 
Let's say the maximum displacement of the spring towards the top (as in (a)) is indicated by a value of 1 and the middle position of the string (e) is a 0.
If we plot the time on the x-axis and the displacement on the y-axis, this would look like this: 
"""

x = [(1/2)*np.pi, (3/4)*np.pi, np.pi, (5/4)*np.pi, (3/2)*np.pi, (7/4)*np.pi, 2*np.pi]
y = np.sin(x)
fig,ax = plt.subplots()
ax.set_xticks(x, labels=["a", "c", "e", "g", "i", "k", "m"])
ax.scatter(x,y, color="black")
fig.set_size_inches(4,2)
output = Output()
with output:
    display(fig)

text_3 = """
If we do this more frequently and with smaller timesteps in between, the curve looks more smooth:
"""

output2 = Output()
x = np.arange((1/2)*np.pi, 2*np.pi, 0.1)
y = np.sin(x)
fig,ax = plt.subplots()
ax.scatter(x,y, color="black")
fig.set_size_inches(4,2)
with output2:
    display(fig)

panel_1 = VBox([HTML(text), im_string_moving, HTML(text_2), output, HTML(text_3), output2])

text = """
Now we saw a moving string turn into a signal. This was possible, because it's movemenet was periodic, i.e. recurring over time. 
Likewise, brain activity can be periodic. Say you measure the activity over time and see that it goes up and down again and again. 
Can you imagine that such a signal would look like this?
"""
output = Output()
x = np.arange((1/2)*np.pi, 12*np.pi, 0.1)
y = np.sin(x)
fig,ax = plt.subplots()
ax.plot(x,y, color="black")
fig.set_size_inches(4,2)
with output:
    display(fig)
panel_2 = VBox([HTML(text), output])

chapter = get_chapter([panel_1, panel_2], ["1", "2"])
display(chapter)

#https://www.phys.unsw.edu.au/jw/strings.html

# (4) Signal Properties

In [82]:

im_1_5 = Image(value=open("resources/amp_1_freq_5.png", "rb").read(),width=width,height=height)
im_1_25 = Image(value=open("resources/amp_1_freq_25.png", "rb").read(),width=width,height=height)
im_7_5 = Image(value=open("resources/amp_7_freq_5.png", "rb").read(),width=width,height=height)
im_7_25 = Image(value=open("resources/amp_7_freq_25.png", "rb").read(),width=width,height=height)




text1 = """
In the previous chapter we understood what a signal is and why brain activity can be interpreted as such. 
We now want to discuss some properties that describe signals that are the <b>Frequency</b> and the <b>Amplitude</b>.
<br>
The <b>frequency</b> indicates, how often a signal oscillates, i.e., how often it goes up and down in a given time-range. We measure the frequency in Hertz (Hz), which can be understood as the number of ups and downs per second. 
A frequency of 50Hz means 50 ups and downs in one second. 
You can use the frequency to describe different processes that repeat over time. For example, you heart beats ~60 times a minute, which is a frequency of 0.016Hz. 
If you play the A-key on a piano, the string and the air it puts into motion will oscilate with a frequency of 440Hz. If you play a higher note, the frequency increases and with lower notes, the frequency decreases. 
Electromagentic waves have frequencys as well. Red light has a frequency of 400THz (terra herz, that is 10¹² herz) and X-rays have a frequency around 10¹⁸ Hz. 
"""

panel_1 = HTML(text1)

text = """
As mentioned, the frequency in Hz is the number of oscilations per second. In the following you see two signals. 
"""
answer_text = """
The left signal has a frequency of 5 Hz. You can find out by counting how often the signal does a full oscilation from 0 (where it starts), up to 1, down to -1 and back to 0.
You could also count the number of mountains (+1) or valleys (-1) in 1 second. 
The right signal has a higher frequency of 25 Hz. 
"""
answer = get_answer_to_reveal("Can you tell, which frequencies they have?", answer_text)
panel_2 = VBox([HTML(text), HBox([im_1_5, im_1_25]), answer])

text = """
The <b>amplitude</b> tells you, how much a signal goes up and down in one cycle. 
In the following you see two plots of same frequency but with different amplitudes. Do you see, that the one covers a range from -1 to +1 on the y-axis, but the other covers a bigger range from -7 to +7?
The first signal has an amplitude of 1, the second has an amplitude of 7. 
"""
text_2 = """
The amplitude of a signal is related to its power (which will be explained later in more detail). If you play a sound on an instrument, higher amplitude means a louder sound. 
A signal with too much power could even destroy your ear. 
"""
panel_3 = VBox([HTML(text), HBox([im_1_5,im_7_5]), HTML(text_2)])

im_4_15 = Image(value=open("resources/amp_4_freq_15.png", "rb").read(),width=width,height=height)
text = """
Test yourself!
"""
answer = get_answer_to_reveal("What is the frequency and the amplitude of this signal?", "Amplitude: 4 (because the signal goes from -4 to 4 on the y-axis). Frequency: 15 (because there are 15 ups and downs within one second).")
panel_4 = VBox([HTML(text), im_4_15, answer])

chapter = get_chapter([panel_1, panel_2, panel_3, panel_4], ["1","2", "3", "Test yourself!"])
display(chapter)

# (5) Sampling Frequency

In [83]:
%matplotlib inline

file = open("resources/10_hz_signal.png", "rb")



text1 = "Say we have a signal that looks like this."
im_signal = Image(value=open("resources/10_hz_signal.png", "rb").read(),width=width,height=height)
panel_1 = VBox([HTML(text1), im_signal])

text2 = "We now sample this signal, i.e., we measure it's vaue at different points in time. These points in time we see as orange dots here. All the points have equal distance on the x-axis. If we do that often enough, we can connect the dots and get the signal."
im_signal_sampled = Image(value=open("resources/10_hz_signal_sampled.png", "rb").read(),width=width,height=height)
panel_2 = VBox([HTML(text2), im_signal_sampled])

text3 = "However, what would happen, if we use very few points only? Take a look at the following example. Again, these points all have the same distance on the x-axis. However, they don't allow to capture the full range of the signal. If we connect them, we get a signal (flat orange line) that is very different from the one, we wanted to sample."
im_signal_sampled_incorrectly_1 = Image(value=open("resources/10_hz_signal_sampled_incorrectly_1.png", "rb").read(),width=width,height=height)
panel_3 = VBox([HTML(text3), im_signal_sampled_incorrectly_1])

text4 = "It would be very bad luck, if all our points catch the exact same value of the signal, as in the previous example. However, if the number of points is too small, we can end up with arbitrary signals, that just don't reflect the signal we are sampling from"
im_signal_sampled_incorrectly_2 = Image(value=open("resources/10_hz_signal_sampled_incorrectly_2.png", "rb").read(),width=width,height=height)
panel_4 = VBox([HTML(text4), im_signal_sampled_incorrectly_2])

text_explanation = "We just saw, that having too feq sampling points can be problematic. The number of sampling points in a given time-range is called the sampling frequency. E.g., if we collect 10 points per second, this is a sampling frequency of 10Hz. \n If we have a signal of n Hz, the sampling frequency must be at least 2*n Hz to capture the signal. This is called the Nyquist Frequency. The other way round, if you sample with a frequency of n Hz, you can not expect to capture signals that have a frequency of more than n/2 Hz."
panel_5 = HTML(text_explanation)


output = Output()
def sample(sampling_freq=50):
    x = np.arange(0, 1, 0.001)
    # create a 20hz signal
    y = np.sin(20*x*2*np.pi)
    x_sample = np.arange(0,1, 1/sampling_freq)
    # sample the 20hz signal with the given sampling_freq
    sample = np.sin(20*x_sample*2*np.pi)
    
    fig,axs = plt.subplots(1,2,sharex=True, sharey=True)
    axs[0].plot(x,y, color="black")
    axs[0].scatter(x_sample, sample, color="orange")
    axs[1].plot(x_sample, sample, color="orange")
    fig.set_size_inches(14,6)
    return fig

def show_sampling_plot(b,sampling_freq=None):
    if sampling_freq is None:
        sampling_freq = int(input_freq.value)
    fig = sample(sampling_freq)
    
    with output:
        clear_output()
        display(fig)
    print("this")

show_sampling_plot(None, sampling_freq=50)
text = "You can try it yourself! The signal you see here has a frequency of 20Hz. What happens if you sample it with lower or higher frequencies?"
input_freq = Text(placeholder="50")
button = Button(description="Sample")
button.on_click(show_sampling_plot)
panel_6 = VBox([HTML(text), HBox([input_freq, button]), output])


test_input_field = Text()
answer = get_answer_to_reveal("You are sampling a signal with a frequency of 80Hz. What is the maximum frequency you can expect in the signal you construct from the sampling points?", "40 Hz")
panel_7 = HBox([test_input_field, answer])





chapter = get_chapter([panel_1, panel_2, panel_3, panel_4, panel_5, panel_6, panel_7], ["1","2","3","4", "Explanation", "Try it yourself", "Test yourself!"])

display(chapter)

this


# (6) EEG bandwidths

In [84]:
#https://nhahealth.com/brainwaves-the-language/
# https://www.researchgate.net/publication/275830679_A_New_EEG_Acquisition_Protocol_for_Biometric_Identification_Using_Eye_Blinking_Signals?_tp=eyJjb250ZXh0Ijp7ImZpcnN0UGFnZSI6Il9kaXJlY3QiLCJwYWdlIjoiX2RpcmVjdCJ9fQ

text = "In the following, we will learn what are typical frequencies for EEG data. We will see that there are different bandwidths, that are associated with different kinds of brain activity."
panel_1 = HTML(text)

text = """
What range of frequencies do EEG waves occur in? We already saw, that signals in general can vary from very fast (X-rays in petaherz range) to very slow (your pulse with ~1 Hz). 
The brain does not cover this whole spectrum. One can say, that brain activity is limited to the range of roughly 0.5 to 100Hz.
As a convention, EEG signals are sub-divided into five different bandwiths Alhpa, Beta, Gamma, Delta and Theta, which include different frequencies. The main reason for this sub-division is, that these bandwidths are associated with different kinds of activity:
<p><b>Delta</b> (less than 4Hz) <br> Delta waves have the lowest frequency and occur during sleep.</p>
<p><b>Theta</b> (4-8 Hz) <br> Theta waves appear somewhere in the border between sleep and an awake state. If you daydreaming or doing medidation, theta waves may increase.</p>
<p><b>Alpha</b> (8-12Hz) <br> Alpha waves reflect a state of alertnes and being prepared to act. However, they are associated with a rather relaxed state and not so much with concentrating deliberately.</p>
<p><b>Beta</b> (13-30Hz) <br> When you are awake and have your eyes open, Beta waves occur. They are associated with conscious activities such as listening or thinking concrentratedly.</p>
<p><b>Gamma</b> (greater than 30 Hz) <br> The fast Gamma waves appear, when you need to process multiple sources of information in parallel.</p>

You can learn more about bandwiths <a href="https://nhahealth.com/brainwaves-the-language/" target="_blank">here</a>

"""
panel_2 = HTML(text)

text = "This is how the bandwidths look like in an EEG signal. Do you agree, that Gamma waves have the highest, and Delta waves have the lowest frequency? Do you think you can specify the correct bandwidth for a given signal? If so, go to the next tab!"
image_source = """<a href="https://www.researchgate.net/publication/275830679_A_New_EEG_Acquisition_Protocol_for_Biometric_Identification_Using_Eye_Blinking_Signals?_tp=eyJjb250ZXh0Ijp7ImZpcnN0UGFnZSI6Il9kaXJlY3QiLCJwYWdlIjoiX2RpcmVjdCJ9fQ" target="_blank">Image source</a>"""
im_frequency_bands = Image(value=open("resources/frequency_bands_eeg.png", "rb").read(),width=width,height=height)
panel_3 = VBox([HTML(text), im_frequency_bands, HTML(image_source)])

text = "Here are two signals. What bandwidths do they belong to?"
panel_4 = HTML(text)

chapter = get_chapter([panel_1, panel_2, panel_3, panel_4], ["1","2", "3", "Test yourself!"])
display(chapter)

# (7) Power Spectrum

In [85]:
%matplotlib inline

#https://neuroimage.usc.edu/forums/t/eeg-power-spectral-density/3634

text = """In this chapter, we will introduce the Power Spectrum, which is a usefull way to display properties of an EEG signal. 
For the power spectrum, we have the frequency on the x-axis and the Power on the y-axis. The power is derived from the squared amplitude of the signal. What does that mean? 
You can understand the amplitude as a measure of the energy. The higher the amplitude, the more energy a signal has. You can easily recognize this, when you hear music. A higher amplitude means a louder sound and that is more energy. 
You don't need to care why exactly we have to square the amplitude (this is just physics), but be aware that squaring a high amplitude also leads to high power.
Summing up, the power is a measure of "how much energy" is in a signal. When we plot this power over the frequency, we get an idea, which different components an EEG signal consists of.
<br>
Let's take a look at such a power spectrum.
"""

im_power_spectrum = Image(value=open("resources/power_spectrum.png", "rb").read(),width=width,height=height)
text_2 = """
Do you see, that there is quite a high power around 10Hz? Looks like the 10Hz waves are contributing more to the overall signal than the 20Hz waves, for example.
If you watched the figure carefully, you may have noticed, that the y-axis is not micro-volt squared, but micro-volt squared divided by Hz. This is done to obtain better comparibilty between the different bandwidths. 
Naturally, waves with higher frequencies contain more energy, which would make it unfair to compare a wave of, say 50Hz with one of 1Hz directly. Dividing by the frequency mitigates this problem. 
If we divide the micro-volt squared by the frequency, we call this a <b>Power Spectrum Density (PSD)</b>. <br>
Sometimes you may also see the y-axis of a power spectrum (density) to be in decibel (db). This is just another way of scaling the power to compare different magnitudes of power more easily.
You can still interpret the plots the same way though: Higher values mean more power. 
"""

panel_1 = VBox([HTML(text), im_power_spectrum, HTML(text_2)])

text = """Remember the bandwidths we saw in the previous chapter? With the power spectrum, it is quite easy to identify which frequencies are present in an EEG signal. Different areas on the x-axis belong to the different bandwidths."""
im_sleep_eeg = Image(value=open("resources/sleep_eeg.png", "rb").read(),width=width,height=height)
im_band_power = Image(value=open("resources/band_power.png", "rb").read(),width=width,height=height)
text_2 = """
 If you see which bandwidth are dominant, you can already get an idea of what the person is doing.
 In the following you see EEGs for people that are awake (top left) or sleeping (remaining three images). Do you see how the alpha waves are spiking in the awake state but decline in the sleep states?
Which channels are more prominent during sleep? Does that match what you learned about the bandwidths in the previous chapter?
"""

panel_2 = VBox([HTML(text), im_band_power, HTML(text_2), im_sleep_eeg])


text = "The power spectrum can be plotted for each channel individually or as an average over each channel."
answer = get_answer_to_reveal("Can you guess, what could be an advantage of showing it for each channel?", "If you display each channel, you can detect if some channels behave different than others. There are different reasons for that, but one could be, that a channel captures elictric signals that are not coming from the brain but are noise we don't want.")
panel_3 = VBox([HTML(text), answer])


text = "Take a look at the power spectrum of your own data. Do you observe anything interesting? Is the power rising or falling for higher frequencies? Are there any frequencies that are dominant?"
psd = raw.compute_psd(fmax=120)
psd_plot = psd.plot(show=False)
output = Output()
with output:
    display(psd_plot)
panel_4 = VBox([HTML(text),output])

chapter = get_chapter([panel_1, panel_2, panel_3, panel_4], ["1","2", "3", "Check your own data"])
display(chapter)

#https://link.springer.com/article/10.1007/s11571-020-09639-w
#https://www.researchgate.net/publication/346510774_Criticality_and_the_role_of_the_connectome_in_shaping_slow_oscillations_in_the_brain_during_deep_sleep?_tp=eyJjb250ZXh0Ijp7ImZpcnN0UGFnZSI6Il9kaXJlY3QiLCJwYWdlIjoiX2RpcmVjdCJ9fQ

Effective window size : 2.000 (s)
Plotting power spectral density (dB=True).


/tmp/ipykernel_15627/3220567632.py:45: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  psd_plot = psd.plot(show=False)
